<h1 style="text-align:center;">Upload Firmware component and associated Component Signature (.comsig) file to iLO repository </h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contains script for uploading a firmware component with a size less than
32 GB, and associated compsig signature file to the iLO Repository.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
        <li>getpass - to take password as inputs securely</li> 
    </ul>
</div>

In [ ]:
import os
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our upload_firmware function overview, in the end this will upload firmware component along with compsig to the iLO Repository.</h3><br>
    
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to do this.<br>

1. This function takes four parameters: <b>_redfishobj</b> which is the Redfish client object, <b>firmware_loc</b> which is the path to the firmware image file, <b>compsig_loc</b> which is the path to the firmware image signature file, <b>update_repo</b> which is a boolean flag to specify whether to update the firmware repository or not, and <b>update_target</b> which is a boolean flag to specify whether to update the target device or not.
<br>
    
2. Call <b>get_resource_directory()</b> function by passing the <b>_redfishobj</b> as a parameter to retrieve the resource directory of the server. If the resource directory is not available or it is disabled, find the relevant URI by navigating through the server's API. If the resource directory is available, find the relevant URI from the resource directory.
    <br>

3. Then the function retrieves the update service URI from the response of the previous step.
    <br>

4. The function reads the firmware and signature files and prepares the data in the required format to upload the firmware using the HttpPushUri. The function builds a list of tuples that contain the session key, json data, firmware file, and firmware signature file.
    <br>

5. The function creates a header dictionary containing the session key.
    <br>

6. The function passes the list of payload to the <b>_redfishobj.post()</b> method with the HttpPushUri as the path to upload the firmware and signature files.
    <br>


7. If the HTTP response status code is 400, print an error message indicating that the firmware upload failed with the HTTP response message. If the HTTP response status code is neither 200 nor 201, print an error message indicating that an HTTP response with the status code was returned. If the HTTP response status code is 200 or 201, print a success message indicating that the firmware upload is complete.
    <br>
</div>

In [ ]:
def upload_firmware(_redfishobj, firmware_loc, compsig_loc, update_repo=True, update_target=False):
    resource_instances = get_resource_directory(_redfishobj)

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #resource directory is not available so we will navigate through paths manually
        update_service_uri = _redfishobj.root.obj['UpdateService']['@odata.id']
    else:
        #obtain all account instances from resource directory
        for instance in resource_instances:
            if '#UpdateService.' in instance['@odata.type']:
                update_service_uri = instance['@odata.id']

    update_service_response = _redfishobj.get(update_service_uri)

    path = update_service_response.obj.HttpPushUri

    body = []
    json_data = {'UpdateRepository': update_repo, 'UpdateTarget': update_target, 'ETag': 'atag', 'Section': 0}
    session_key = _redfishobj.session_key

    filename = os.path.basename(firmware_loc)
    with open(firmware_loc, 'rb') as fle:
        output = fle.read()

    compsigname = os.path.basename(compsig_loc)
    with open(compsig_loc, 'rb') as cle:
        compsigoutput = cle.read()


    session_tuple = ('sessionKey', session_key)
    parameters_tuple = ('parameters', json.dumps(json_data))
    file_tuple = ('file', (filename, output, 'application/octet-stream'))
    compsig_tuple = ('compsig', (compsigname, compsigoutput, 'application/octet-stream'))

    #Build the payload from each multipart-form data tuple
    body.append(session_tuple)
    body.append(parameters_tuple)
    body.append(compsig_tuple)
    body.append(file_tuple)

    #Create our header dictionary
    header = {'Cookie': 'sessionKey=' + session_key}

    #We pass the whole list payload to post
    resp = _redfishobj.post(path, body, headers=header)

    if resp.status == 400:
        sys.stderr.write("Failed to upload firmware... Error: '%s'\n" % str(resp))
    elif not resp.status in [200, 201]:
        sys.stderr.write("An http response of '%s' was returned.\n" % resp.status)
    else:
        print("Upload complete!\n")


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to upload new firmware to iLO repository for flashing.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> class takes 3 parameters<i> iLO url, Username, Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
    <br>

4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
    <br>
5. Then we call <b>upload_firmware</b> with our redfish object, FIRMWARE_PATH, COMPSIG_PATH, UPDATE_REPO (set as True) and UPDATE_TARGET (set as False) as parameters.<br>
    

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.<br><br>
    
7. <b>When prompted for entering Firmware companent path, enter the entire path containing the firmware component.</b>. Example: "/path/to/cpxxxxxx.exe"<br>The path to the firmware component file to upload. .rpm and .zip component are also valid.<br><br>
    
8. <b>When prompted for entering compsig path, enter the entire path containing the Component Signature (compsig) file having .compsig extension.</b>. Example: "/path/to//cpxxxxxx.compsig"<br><br>

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # The path to the firmware component file to upload.
    # .rpm and .zip component are also valid.
    #
    
    FIRMWARE_PATH = input("Enter firmware path ")
    COMPSIG_PATH = input("Enter compsig path ")
    # Upload the firmware file to the iLO Repository
    UPDATE_REPO = True
    # Update the system with the firmware file
    UPDATE_TARGET = False

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    upload_firmware(REDFISHOBJ, FIRMWARE_PATH, COMPSIG_PATH, UPDATE_REPO, UPDATE_TARGET)

    REDFISHOBJ.logout()


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help please check the follwoing links for reference:
    <br>
    1. Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">Python ilorest library</a><br>
    2. HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">ilorestful api explorer</a><br>
    3. HPE iLO Redfish Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/">HPE iLO Redfish API Documentation</a><br>
    4. REST API: <a href = "https://restfulapi.net/">Restful API</a><br>    
</div>